In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size by factor 2
        #Shape= (256,12,75,75)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)

        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)

        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)



    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)


            #Above output will be in matrix form, with shape (256,32,75,75)

        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output

In [ ]:
num_epochs=10
learning_rate=0.001

## Spiral

In [ ]:
spiral_train_path='drive/MyDrive/parkinsons/Spiral'
spiral_test_path='drive/MyDrive/parkinsons/Test/Test Spiral'

In [ ]:
spiral_train_loader=DataLoader(
    torchvision.datasets.ImageFolder(spiral_train_path,transform=transformer),
    batch_size=64, shuffle=True
)

spiral_test_loader=DataLoader(
    torchvision.datasets.ImageFolder(spiral_test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
root=pathlib.Path(spiral_train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print("Train classes: " + str(classes))

root_t=pathlib.Path(spiral_test_path)
classes=sorted([j.name.split('/')[-1] for j in root_t.iterdir()])
print("Test classes:  " + str(classes))

Train classes: ['HealthySpiral', 'PatientSpiral']
Test classes:  ['HealthySpiral', 'PatientSpiral']


In [ ]:
train_count=len(glob.glob(spiral_train_path+'/**/*.jpg'))
test_count=len(glob.glob(spiral_test_path+'/**/*.jpg'))

print(train_count,test_count)

248 20


In [ ]:
spiral_model=ConvNet(num_classes=2).to(device)

optimizer=Adam(spiral_model.parameters(),lr=learning_rate,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    spiral_model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(spiral_train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=spiral_model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count




    spiral_model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(spiral_test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=spiral_model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))




    if test_accuracy>best_accuracy:
        torch.save(spiral_model.state_dict(),'best_checkpoint.spiral_model')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(7.1405) Train Accuracy: 0.5725806451612904 Test Accuracy: 0.5
Epoch: 1 Train Loss: tensor(4.4622) Train Accuracy: 0.842741935483871 Test Accuracy: 0.65
Epoch: 2 Train Loss: tensor(3.0238) Train Accuracy: 0.8790322580645161 Test Accuracy: 0.8
Epoch: 3 Train Loss: tensor(1.7356) Train Accuracy: 0.9032258064516129 Test Accuracy: 0.85
Epoch: 4 Train Loss: tensor(1.2199) Train Accuracy: 0.9314516129032258 Test Accuracy: 0.8
Epoch: 5 Train Loss: tensor(0.7668) Train Accuracy: 0.9516129032258065 Test Accuracy: 0.8
Epoch: 6 Train Loss: tensor(0.3514) Train Accuracy: 0.9516129032258065 Test Accuracy: 0.75
Epoch: 7 Train Loss: tensor(0.2768) Train Accuracy: 0.9717741935483871 Test Accuracy: 0.8
Epoch: 8 Train Loss: tensor(0.0600) Train Accuracy: 0.9959677419354839 Test Accuracy: 0.9
Epoch: 9 Train Loss: tensor(0.2322) Train Accuracy: 0.9637096774193549 Test Accuracy: 0.9


# Meander

In [ ]:
meander_train_path='drive/MyDrive/parkinsons/Meander'
meander_test_path='drive/MyDrive/parkinsons/Test/Test Meander'

In [ ]:
meander_train_loader=DataLoader(
    torchvision.datasets.ImageFolder(meander_train_path,transform=transformer),
    batch_size=64, shuffle=True
)

meander_test_loader=DataLoader(
    torchvision.datasets.ImageFolder(meander_test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
root=pathlib.Path(meander_train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print("Train classes: " + str(classes))

root_t=pathlib.Path(meander_test_path)
classes=sorted([j.name.split('/')[-1] for j in root_t.iterdir()])
print("Test classes:  " + str(classes))

In [ ]:
train_count=len(glob.glob(meander_train_path+'/**/*.jpg'))
test_count=len(glob.glob(meander_test_path+'/**/*.jpg'))

print(train_count,test_count)

In [ ]:
meander_model=ConvNet(num_classes=2).to(device)

optimizer=Adam(meander_model.parameters(),lr=learning_rate,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    meander_model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(meander_train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=meander_model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count




    meander_model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(meander_test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=meander_model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))




    if test_accuracy>best_accuracy:
        torch.save(meander_model.state_dict(),'best_checkpoint.meander_model')
        best_accuracy=test_accuracy

# Circle

In [ ]:
circle_train_path='drive/MyDrive/parkinsons/Circle'
circle_test_path='drive/MyDrive/parkinsons/Test/Test Circle'

In [ ]:
circle_train_loader=DataLoader(
    torchvision.datasets.ImageFolder(circle_train_path,transform=transformer),
    batch_size=64, shuffle=True
)

circle_test_loader=DataLoader(
    torchvision.datasets.ImageFolder(circle_test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
root=pathlib.Path(circle_train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print("Train classes: " + str(classes))

root_t=pathlib.Path(circle_test_path)
classes=sorted([j.name.split('/')[-1] for j in root_t.iterdir()])
print("Test classes:  " + str(classes))

In [ ]:
train_count=len(glob.glob(circle_train_path+'/**/*.jpg'))
test_count=len(glob.glob(circle_test_path+'/**/*.jpg'))

print(train_count,test_count)

In [ ]:
circle_model=ConvNet(num_classes=2).to(device)

optimizer=Adam(circle_model.parameters(),lr=learning_rate,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    circle_model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(circle_train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=circle_model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count




    circle_model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(circle_test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=circle_model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))




    if test_accuracy>best_accuracy:
        torch.save(circle_model.state_dict(),'best_checkpoint.circle_model')
        best_accuracy=test_accuracy